# Here a section for pre-processing data scraped

In [1]:
import json
import pandas as pd
from datetime import datetime
import spacy
import re


In [2]:
# Charger le fichier JSON
with open("data_scraped.json", "r", encoding="utf-8") as f:
    data = json.load(f)

rows = []

for publication in data:
    # Nettoyage du texte de la publication
    post_text = "\n".join([p.strip() for p in publication.get("post", []) if p.strip()])
    
    # Format date (facultatif)
    raw_date = publication.get("date_publication", "").replace("Publié le ", "")
    
    # Ajouter la publication principale
    rows.append({
        "type": "publication",
        "date": raw_date,
        "texte": post_text,
        "source_url": publication.get("url", "")
    })

    # Ajouter les commentaires et replies
    for comment in publication.get("comments", []):
        comment_text = comment.get("text", "").strip()
        comment_date = comment.get("date", "").strip()
        
        rows.append({
            "type": "comment",
            "date": comment_date,
            "texte": comment_text,
            "source_url": publication.get("url", "")
        })

        for reply in comment.get("replies", []):
            reply_text = reply.get("text", "").strip()
            reply_date = reply.get("date", "").strip()
            
            rows.append({
                "type": "reply",
                "date": reply_date,
                "texte": reply_text,
                "source_url": publication.get("url", "")
            })

# Convertir en DataFrame
df = pd.DataFrame(rows)

In [3]:
df

,type,date,texte,source_url
0,publication,dimanche 6 avril 2025 à 22h57min,Il fut une époque à Pouni où l’expression « Ke...,https://lefaso.net/spip.php?article137228
1,comment,6 avril 08:22,"Heu, si j’ai bien compris, nos autorités actue...",https://lefaso.net/spip.php?article137228
2,comment,6 avril 10:58,Bravo à cet intellectuel de haut vol pour qui ...,https://lefaso.net/spip.php?article137228
3,publication,dimanche 6 avril 2025 à 23h00min,« L’administration de la justice dans un conte...,https://lefaso.net/spip.php?article137236
4,publication,jeudi 3 avril 2025 à 23h17min,"(Ouagadougou, 03 avril 2025). Le Président du ...",https://lefaso.net/spip.php?article137191
...,...,...,...,...
15802,reply,14 février 2024 13:51,"@Patrick Kibora, bon résumé de la situation du...",https://lefaso.net/spip.php?article127965
15803,comment,15 février 2024 12:41,"@Patrick Kibora\n\nMessire Patrick Kibora, je ...",https://lefaso.net/spip.php?article127965
15804,publication,mardi 13 février 2024 à 20h45min,La dégradation de la situation socio-politique...,https://lefaso.net/spip.php?article127980
15805,comment,14 février 2024 20:34,"Tous les pays, toutes les organisations qui on...",https://lefaso.net/spip.php?article127980


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15807 entries, 0 to 15806
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   type        15807 non-null  object
 1   date        15807 non-null  object
 2   texte       15807 non-null  object
 3   source_url  15807 non-null  object
dtypes: object(4)
memory usage: 494.1+ KB


## I-Pre-Processing

In [3]:
!pip install spacy

In [4]:
!python -m spacy download fr_core_news_sm

     ---------------------------------------- 0.0/16.3 MB ? eta -:--:--
      --------------------------------------- 0.3/16.3 MB 6.7 MB/s eta 0:00:03
     - -------------------------------------- 0.7/16.3 MB 8.9 MB/s eta 0:00:02
     --- ------------------------------------ 1.6/16.3 MB 12.7 MB/s eta 0:00:02
     ----- ---------------------------------- 2.4/16.3 MB 12.9 MB/s eta 0:00:02
     ----- ---------------------------------- 2.4/16.3 MB 12.9 MB/s eta 0:00:02
     --------- ------------------------------ 3.8/16.3 MB 13.4 MB/s eta 0:00:01
     ------------ --------------------------- 5.0/16.3 MB 15.2 MB/s eta 0:00:01
     ------------ --------------------------- 5.1/16.3 MB 14.9 MB/s eta 0:00:01
     -------------- ------------------------- 5.8/16.3 MB 14.2 MB/s eta 0:00:01
     -------------- ------------------------- 5.8/16.3 MB 14.2 MB/s eta 0:00:01
     -------------- ------------------------- 5.8/16.3 MB 14.2 MB/s eta 0:00:01
     -------------- ------------------------- 5.8/

In [5]:
# Model loading
nlp = spacy.load("fr_core_news_sm")

In [6]:
def preprocess_text(text):
    doc = nlp(text)

    tokens = [
        token.text.lower() 
        for token in doc 
        if not token.is_punct and not token.is_space and not re.search(r'[@#]', token.text)
    ]
    
    return " ".join(tokens)

In [7]:
df["text_processed"] = df["texte"].apply(preprocess_text) #preprocess of text

In [8]:
df

,type,date,texte,source_url,text_processed
0,publication,dimanche 6 avril 2025 à 22h57min,Il fut une époque à Pouni où l’expression « Ke...,https://lefaso.net/spip.php?article137228,il fut une époque à pouni où l’ expression ken...
1,comment,6 avril 08:22,"Heu, si j’ai bien compris, nos autorités actue...",https://lefaso.net/spip.php?article137228,heu si j’ ai bien compris nos autorités actuel...
2,comment,6 avril 10:58,Bravo à cet intellectuel de haut vol pour qui ...,https://lefaso.net/spip.php?article137228,bravo à cet intellectuel de haut vol pour qui ...
3,publication,dimanche 6 avril 2025 à 23h00min,« L’administration de la justice dans un conte...,https://lefaso.net/spip.php?article137236,l’ administration de la justice dans un contex...
4,publication,jeudi 3 avril 2025 à 23h17min,"(Ouagadougou, 03 avril 2025). Le Président du ...",https://lefaso.net/spip.php?article137191,ouagadougou 03 avril 2025 le président du faso...
...,...,...,...,...,...
15802,reply,14 février 2024 13:51,"@Patrick Kibora, bon résumé de la situation du...",https://lefaso.net/spip.php?article127965,kibora bon résumé de la situation du congo bel...
15803,comment,15 février 2024 12:41,"@Patrick Kibora\n\nMessire Patrick Kibora, je ...",https://lefaso.net/spip.php?article127965,kibora messire patrick kibora je ne sais si vo...
15804,publication,mardi 13 février 2024 à 20h45min,La dégradation de la situation socio-politique...,https://lefaso.net/spip.php?article127980,la dégradation de la situation socio-politique...
15805,comment,14 février 2024 20:34,"Tous les pays, toutes les organisations qui on...",https://lefaso.net/spip.php?article127980,tous les pays toutes les organisations qui ont...


#### Normalization of Date to ISO format

In [11]:
!pip install dateparser

In [9]:
import datetime
import dateparser


def normaliser_date(date_str, reference_year=None):
    # Nettoyage manuel
    clean_date = date_str.lower()
    clean_date = re.sub(r'publié le ', '', clean_date)
    clean_date = re.sub(r'à ', '', clean_date)
    clean_date = re.sub(r'min', '', clean_date)
    clean_date = clean_date.strip()

    # Settings pour dateparser
    settings = {'PREFER_DATES_FROM': 'past', 'DATE_ORDER': 'DMY'}

    if reference_year:
        settings['RELATIVE_BASE'] = datetime.datetime(reference_year, 1, 1)

    parsed = dateparser.parse(clean_date, settings=settings)

    if parsed:
        return parsed.strftime("%Y-%m-%d %H:%M:%S")
    else:
        return None

In [10]:
# Application à une série de dates
df["date_normalized"] = df["date"].apply(lambda x: normaliser_date(x, reference_year=2025))

In [11]:
df

,type,date,texte,source_url,text_processed,date_normalized
0,publication,dimanche 6 avril 2025 à 22h57min,Il fut une époque à Pouni où l’expression « Ke...,https://lefaso.net/spip.php?article137228,il fut une époque à pouni où l’ expression ken...,2025-04-06 22:57:00
1,comment,6 avril 08:22,"Heu, si j’ai bien compris, nos autorités actue...",https://lefaso.net/spip.php?article137228,heu si j’ ai bien compris nos autorités actuel...,2024-04-06 08:22:00
2,comment,6 avril 10:58,Bravo à cet intellectuel de haut vol pour qui ...,https://lefaso.net/spip.php?article137228,bravo à cet intellectuel de haut vol pour qui ...,2024-04-06 10:58:00
3,publication,dimanche 6 avril 2025 à 23h00min,« L’administration de la justice dans un conte...,https://lefaso.net/spip.php?article137236,l’ administration de la justice dans un contex...,2025-04-06 23:00:00
4,publication,jeudi 3 avril 2025 à 23h17min,"(Ouagadougou, 03 avril 2025). Le Président du ...",https://lefaso.net/spip.php?article137191,ouagadougou 03 avril 2025 le président du faso...,2025-04-03 23:17:00
...,...,...,...,...,...,...
15802,reply,14 février 2024 13:51,"@Patrick Kibora, bon résumé de la situation du...",https://lefaso.net/spip.php?article127965,kibora bon résumé de la situation du congo bel...,2024-02-14 13:51:00
15803,comment,15 février 2024 12:41,"@Patrick Kibora\n\nMessire Patrick Kibora, je ...",https://lefaso.net/spip.php?article127965,kibora messire patrick kibora je ne sais si vo...,2024-02-15 12:41:00
15804,publication,mardi 13 février 2024 à 20h45min,La dégradation de la situation socio-politique...,https://lefaso.net/spip.php?article127980,la dégradation de la situation socio-politique...,2024-02-13 20:45:00
15805,comment,14 février 2024 20:34,"Tous les pays, toutes les organisations qui on...",https://lefaso.net/spip.php?article127980,tous les pays toutes les organisations qui ont...,2024-02-14 20:34:00


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15807 entries, 0 to 15806
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   type             15807 non-null  object
 1   date             15807 non-null  object
 2   texte            15807 non-null  object
 3   source_url       15807 non-null  object
 4   text_processed   15807 non-null  object
 5   date_normalized  15807 non-null  object
dtypes: object(6)
memory usage: 741.1+ KB


In [13]:
df.to_csv("data_processed/dataset_nlp.csv", index=False, encoding="utf-8")